# Auto-Completion using N-Gram Models

In [1]:
!pip install opendatasets -q

In [2]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/crmercado/tweets-blogs-news-swiftkey-dataset-4million")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: seshupavan
Your Kaggle Key: ··········


100%|██████████| 1.10G/1.10G [00:12<00:00, 98.2MB/s]


In [3]:
import math
import nltk
import random
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

file_path = '/content/tweets-blogs-news-swiftkey-dataset-4million/final/en_US/en_US.twitter.txt'
data_dir = '/path/to/nltk_data'
nltk.data.path.append(data_dir)
nltk.download('punkt')

with open(file_path, "r") as f:
    data = f.read()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### Data Cleaning

In [4]:
def preprocess_pipeline(data):
    "Pre-processes and tokenizes the data"
    sentences = data.split('\n')
    sentences = [s.strip() for s in sentences]
    sentences = [s for s in sentences if len(s) > 0]
    tokenized = []
    for sentence in sentences:
        sentence = sentence.lower()
        token = nltk.word_tokenize(sentence)
        tokenized.append(token)
    return tokenized

tokenized_sentences = preprocess_pipeline(data)

In [6]:
train_data, test_data = train_test_split(tokenized_sentences, test_size=0.2, random_state=42)

### Data Pre-processing

In [5]:
def count_the_words(sentences):
    "Counts the occurance of the words in the corpus and returns key-value pairs"
    word_counts = {}
    for sentence in sentences:
        for token in sentence:
            if token not in word_counts.keys():
                word_counts[token] = 1
            else:
                word_counts[token] += 1
    return word_counts

def handling_oov(tokenized_sentences, count_threshold):
    "Find the words that appears n-times or more"
    closed_vocabulary = []
    words_count = count_the_words(tokenized_sentences)
    for word, count in words_count.items():
        if count >= count_threshold:
            closed_vocabulary.append(word)
    return closed_vocabulary

def replace_oov_words_by_unk(tokenized_sentences, vocabulary, unknown_token="<unk>"):
    "Replaces the unknown words with <unk> token"
    vocabulary = set(vocabulary)
    new_tokenized_words = []
    for sentence in tokenized_sentences:
        replaced_sentence = []
        for token in sentence:
            if token in vocabulary:
                replaced_sentence.append(token)
            else:
                replaced_sentence.append(unknown_token)
        new_tokenized_words.append(replaced_sentence)
    return new_tokenized_words

In [7]:
def preprocess_data(train_data, test_data, count_threshold):
    vocabulary = handling_oov(train_data, count_threshold)
    train_data_replaced = replace_oov_words_by_unk(train_data,vocabulary)
    test_data_replaced = replace_oov_words_by_unk(test_data,vocabulary)
    return train_data_replaced, test_data_replaced, vocabulary

min_freq = 3
final_train, final_test, vocabulary = preprocess_data(train_data, test_data, min_freq)

In [8]:
#Sample data
print(f"The length of training data is {len(final_train)} and first preprocessed sample is:")
print(final_train[0])
print(f"The length of testing data is {len(final_test)} and first preprocessed sample is:")
print(final_test[0])

The length of training data is 1888118 and first preprocessed sample is:
['madison', 'would', 'go', 'into', 'a', 'coma', 'reading', 'what', 'i', 'say', 'about', 'him', '...']
The length of testing data is 472030 and first preprocessed sample is:
['seems', 'difficult', 'to', 'find', 'people', 'on', 'twitter', '.', 'found', 'nytimes', ',', '<unk>', ',', 'but', 'mainly', 'hit', 'and', 'miss', '...', 'any', 'suggestions', 'for', 'finding', 'content', 'sources']


### Building N-gram model

In [14]:
def count_n_grams(data, n, start_token='<s>', end_token = '<e>'):
    n_grams = {}
    for sentence in data:
        sentence = [start_token]*n + sentence + [end_token]
        sentence = tuple(sentence)
        m = len(sentence) if n == 1 else len(sentence) - 1
        for i in range(m):
            n_gram = sentence[i:i+n]
            if n_gram in n_grams.keys():
                n_grams[n_gram] += 1
            else:
                n_grams[n_gram] = 1
    return n_grams

#test-code
sentences = [["Today", "is", "sunday"], ["sunday", "is", "holiday"]]
print(f"unigram, {count_n_grams(sentences, 1)}")
print(f"Bigram, {count_n_grams(sentences, 2)}")

unigram, {('<s>',): 2, ('Today',): 1, ('is',): 2, ('sunday',): 2, ('<e>',): 2, ('holiday',): 1}
Bigram, {('<s>', '<s>'): 2, ('<s>', 'Today'): 1, ('Today', 'is'): 1, ('is', 'sunday'): 1, ('sunday', '<e>'): 1, ('<s>', 'sunday'): 1, ('sunday', 'is'): 1, ('is', 'holiday'): 1, ('holiday', '<e>'): 1}


In [15]:
def estimate_probability(word, previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):
    "calculate probability of a next word using n-gram counts with k-smooting"
    previous_n_gram = tuple(previous_n_gram)
    previous_n_gram_count = n_gram_counts[previous_n_gram] if previous_n_gram in n_gram_counts else 0
    denominator = previous_n_gram_count + k * vocabulary_size
    n_plus1_gram = previous_n_gram + (word,)
    n_plus1_gram_count = n_plus1_gram_counts[n_plus1_gram] if n_plus1_gram in n_plus1_gram_counts else 0
    numerator = n_plus1_gram_count + k
    probability = numerator / denominator
    return probability

def estimate_probabilities(previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0):
    "Estimate the probabilities of next words using the n-gram counts with k-smoothing"
    previous_n_gram = tuple(previous_n_gram)
    vocabulary = vocabulary + ["<e>", "<unk>"]
    vocabulary_size = len(vocabulary)
    probabilities = {}
    for word in vocabulary:
        probability = estimate_probability(word, previous_n_gram,
                                           n_gram_counts, n_plus1_gram_counts,
                                           vocabulary_size, k=k)
        probabilities[word] = probability
    return probabilities

### The Auto Complete System

In [23]:
def suggest_a_word(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0, start_with=None):
    "Suggest a next word based on the probability"
    n = len(list(n_gram_counts.keys())[0])
    previous_n_gram = previous_tokens[-n:]
    probabilities = estimate_probabilities(previous_n_gram,
                                           n_gram_counts, n_plus1_gram_counts,
                                           vocabulary, k=k)
    suggestions = None
    max_prob = 0
    for word, prob in probabilities.items():
        if start_with:
            if not word.startswith(start_with):
                continue
        if prob > max_prob:
            suggestion = word
            max_prob = prob
    return suggestion, max_prob


def get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with=None):
    model_counts = len(n_gram_counts_list)
    suggestions = []
    for i in range(model_counts-1):
        n_gram_counts = n_gram_counts_list[i]
        n_plus1_gram_counts = n_gram_counts_list[i+1]
        suggestion = suggest_a_word(previous_tokens, n_gram_counts,
                                    n_plus1_gram_counts, vocabulary,
                                    k=k, start_with=start_with)
        suggestions.append(suggestion)
    return suggestion

#Testing

In [17]:
n_gram_counts_list = []
for n in range(1, 6):
    n_model_counts = count_n_grams(final_train, n)
    n_gram_counts_list.append(n_model_counts)

In [24]:
previous_tokens = ["hey", "how", "are", "you", "today"]
suggestions_01 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)
print(f"The previous words are {previous_tokens}, the suggestions are:")
display(suggestions_01)

The previous words are ['hey', 'how', 'are', 'you', 'today'], the suggestions are:


('?', 0.0007301070494795521)

In [31]:
previous_tokens = ["hey", "how", "are"]
suggestion_02 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)
print(f"The previous words are {previous_tokens}, the suggestions are:")
display(suggestion_02)

The previous words are ['hey', 'how', 'are'], the suggestions are:


('madison', 9.87615304086752e-06)